# 1. Import Libraries
- Load all necessary libraries (e.g., pandas, numpy, matplotlib, seaborn).

In [1007]:
# import all necessary library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sklearn
import re
from dateutil import parser
from sklearn.preprocessing import MultiLabelBinarizer
import unidecode
import unicodedata
import math

# 2. Load Dataset
- Read the dataset from source (CSV, Excel, SQL, etc.).
- Display the first few rows for inspection.

In [1008]:
# load dataset
# CSV uses semicolon delimiter and non-UTF8 encoding (common in scraped datasets)

df = pd.read_csv(
    "../data/raw/messy_IMDB_dataset.csv",
    sep=";",
    encoding="latin1"
)

# 3. General Overview
- Check dataset dimensions.
- Review column names and data types.
- Generate summary statistics.

In [1009]:
# Check number of rows and columns
print(df.shape, end="\n")

# Preview first 5 rows
df.head()

(101, 12)


,IMBD title ID,Original titlÊ,Release year,Genrë¨,Duration,Country,Content Rating,Director,Unnamed: 8,Income,Votes,Score
0,tt0111161,The Shawshank Redemption,1995-02-10,Drama,142,USA,R,Frank Darabont,NaN,$ 28815245,2.278.845,9.3
1,tt0068646,The Godfather,09 21 1972,"Crime, Drama",175,USA,R,Francis Ford Coppola,NaN,$ 246120974,1.572.674,9.2
2,tt0468569,The Dark Knight,23 -07-2008,"Action, Crime, Drama",152,US,PG-13,Christopher Nolan,NaN,$ 1005455211,2.241.615,9.
3,tt0071562,The Godfather: Part II,1975-09-25,"Crime, Drama",220,USA,R,Francis Ford Coppola,NaN,"$ 4o8,035,783",1.098.714,"9,.0"
4,tt0110912,Pulp Fiction,1994-10-28,"Crime, Drama",,USA,R,Quentin Tarantino,NaN,$ 222831817,1.780.147,"8,9f"


In [1010]:
# Standardize column headers

df = df.rename(columns={
    'IMBD title ID': 'imdb_title_id',
    'Original titlÊ': 'original_title',
    'Release year': 'release_year',
    'Genrë¨': 'genre',
    'Duration': 'duration',
    'Country': 'country',
    'Content Rating': 'content_rating',
    'Director': 'director',
    'Unnamed: 8': 'unnamed_8', 
    'Income': 'income',
    ' Votes ': 'votes',
    'Score': 'score'
})

print(df.columns)

Index(['imdb_title_id', 'original_title', 'release_year', 'genre', 'duration',
       'country', 'content_rating', 'director', 'unnamed_8', 'income', 'votes',
       'score'],
      dtype='object')


In [1011]:
# Check data types and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imdb_title_id   100 non-null    object 
 1   original_title  100 non-null    object 
 2   release_year    100 non-null    object 
 3   genre           100 non-null    object 
 4   duration        99 non-null     object 
 5   country         100 non-null    object 
 6   content_rating  77 non-null     object 
 7   director        100 non-null    object 
 8   unnamed_8       0 non-null      float64
 9   income          100 non-null    object 
 10  votes           100 non-null    object 
 11  score           100 non-null    object 
dtypes: float64(1), object(11)
memory usage: 9.6+ KB


In [1012]:
# Drop the 'unnamed_8' column (empty data)
df = df.drop(columns=['unnamed_8'])

In [1013]:
# Identify rows where all values are NaN
empty_rows = df[df.isna().all(axis=1)]
print("\nRows where all values are NaN:")
print(empty_rows)


Rows where all values are NaN:
   imdb_title_id original_title release_year genre duration country  \
13           NaN            NaN          NaN   NaN      NaN     NaN   

   content_rating director income votes score  
13            NaN      NaN    NaN   NaN   NaN  


In [1014]:
# Drop rows where all values are NaN
df = df.dropna(how="all")

# Recheck number of rows and columns
print(df.shape, end="\n")

(100, 11)


In [1015]:
# Check number of duplicated rows
print("number of duplicated rows: ", df.duplicated().sum())

# Check the number of missing values per column
print("Number of missing values: ")
print(df.isnull().sum())

# Check total missing values across all dataframes
print("Total missing values:", df.isnull().sum().sum())

number of duplicated rows:  0
Number of missing values: 
imdb_title_id      0
original_title     0
release_year       0
genre              0
duration           1
country            0
content_rating    23
director           0
income             0
votes              0
score              0
dtype: int64
Total missing values: 24


In [1016]:
# Statistical summary of the data
df.describe(include="object")

,imdb_title_id,original_title,release_year,genre,duration,country,content_rating,director,income,votes,score
count,100,100,100,100,99,100,77,100,100,100,100
unique,100,100,99,59,71,18,7,64,100,100,28
top,tt0111161,The Shawshank Redemption,2000-05-19,Drama,119,USA,R,Christopher Nolan,$ 28815245,2.278.845,8.6
freq,1,1,2,9,3,62,45,6,1,1,11


# 4. Column-by-Column Cleaning

### 4.1 imdb_title_id

In [1017]:
# Regex pattern for IMDb ID: 'tt' followed by digits (minimum 7 numbers)
pattern = r'^tt\d{7,}$'

# Find rows that do not match the format
outliers = df[~df['imdb_title_id'].astype(str).str.match(pattern)]

print("Number of outliers:", outliers.shape[0])
print(outliers[['imdb_title_id']])


Number of outliers: 0
Empty DataFrame
Columns: [imdb_title_id]
Index: []


### 4.2 original_title

In [1018]:
# 1. Check for missing values in 'original_title'
missing_titles = df[df['original_title'].isnull()]
print("Number of missing titles:", missing_titles.shape[0])

# 2. Check for non-string values in 'original_title'
non_string_titles = df[~df['original_title'].apply(lambda x: isinstance(x, str))]
print("Number of non-string titles:", non_string_titles.shape[0])

# 3. Check for titles that are too short (e.g., single character or empty string)
short_titles = df[df['original_title'].str.len() < 2]
print("Number of suspiciously short titles:", short_titles.shape[0])

# 4. Check for unusual titles (e.g., only numbers or symbols)
weird_titles = df[df['original_title'].str.match(r'^[^A-Za-z]+$', na=False)]
print("Number of unusual titles:", weird_titles.shape[0])
print(weird_titles[['original_title']])

# 5. Check for common mojibake patterns in 'original_title'
# Typical artifacts: Ã, Â, �, mis-decoded byte sequences like B9, C3, A9, etc.
mojibake_pattern = r'(Ã|Â|�|B9|C3|A9|piÃ|AmÃ|LÃ)'
df['mojibake_match'] = df['original_title'].str.extract(mojibake_pattern)

# Filter rows where a mojibake substring was found
mojibake_issues = df[df['mojibake_match'].notnull()]

print("Number of titles with potential mojibake issues:", mojibake_issues.shape[0])
print(mojibake_issues[['original_title', 'mojibake_match']])

Number of missing titles: 0
Number of non-string titles: 0
Number of suspiciously short titles: 0
Number of unusual titles: 1
   original_title
90           1917
Number of titles with potential mojibake issues: 5
                          original_title mojibake_match
27                      La vita B9 bella             B9
36                                 LÃ©on             LÃ
53                               WALLÂ·E              Â
85  Le fabuleux destin d'AmÃ©lie Poulain            AmÃ
98           Per qualche dollaro in piÃ¹            piÃ


In [1019]:
# Function to fix encoding problems
def fix_encoding(text):
    """
    Attempt to repair common text encoding issues (mojibake) by re-encoding 
    from Latin-1 and decoding to UTF-8.

    Parameters
    ----------
    text : str or NaN
        The input text value to check and fix. Can be a string or NaN.

    Returns
    -------
    str or NaN
        The corrected text if encoding repair is successful.
        If the input is NaN or cannot be fixed, the original value is returned.
    """
    if pd.isna(text):
        return text
    try:
        return text.encode('latin1').decode('utf-8')
    except:
        return text

In [1020]:
# Apply encoding fix to create a "fixed original" column
df['original_title_fixed'] = df['original_title'].apply(fix_encoding)

# Strip leading/trailing spaces
df['original_title_fixed'] = df['original_title_fixed'].str.strip()

In [1021]:
# Function to create an analysis-ready snake_case version
def to_snake_case(text):
    """
    Convert a text string into an analysis-ready snake_case format.

    Parameters
    ----------
    text : str or NaN
        The input text value to normalize. Can be a string or NaN.

    Returns
    -------
    str or NaN
        The transformed text in snake_case format:
        - Lowercased
        - Non-alphanumeric characters replaced with underscores
        - Leading/trailing underscores removed
        If the input is NaN, the original value is returned.
    """
    if pd.isna(text):
        return text
    # Lowercase
    text = text.lower()
    # Replace non-alphanumeric characters with underscores
    text = re.sub(r'[^a-z0-9]+', '_', text)
    # Remove leading/trailing underscores
    text = text.strip('_')
    return text

In [1022]:
# Apply snake_case transformation
df['title_clean'] = df['original_title_fixed'].apply(to_snake_case)

# Example check
print(df[['original_title', 'original_title_fixed', 'title_clean']].head(10))

                                  original_title  \
0                       The Shawshank Redemption   
1                                  The Godfather   
2                                The Dark Knight   
3                         The Godfather: Part II   
4                                   Pulp Fiction   
5  The Lord of the Rings: The Return of the King   
6                               Schindler's List   
7                                   12 Angry Men   
8                                      Inception   
9                                     Fight Club   

                            original_title_fixed  \
0                       The Shawshank Redemption   
1                                  The Godfather   
2                                The Dark Knight   
3                         The Godfather: Part II   
4                                   Pulp Fiction   
5  The Lord of the Rings: The Return of the King   
6                               Schindler's List   
7          

In [1023]:
# Drop the 'original_title' and 'mojibake_match' column
df = df.drop(columns=['original_title', 'mojibake_match'])

print(df.head())

  imdb_title_id  release_year                 genre duration country  \
0     tt0111161    1995-02-10                 Drama      142     USA   
1     tt0068646    09 21 1972          Crime, Drama      175     USA   
2     tt0468569   23 -07-2008  Action, Crime, Drama      152      US   
3     tt0071562    1975-09-25          Crime, Drama      220     USA   
4     tt0110912    1994-10-28          Crime, Drama              USA   

  content_rating              director         income      votes score  \
0              R        Frank Darabont     $ 28815245  2.278.845   9.3   
1              R  Francis Ford Coppola    $ 246120974  1.572.674   9.2   
2          PG-13     Christopher Nolan   $ 1005455211  2.241.615    9.   
3              R  Francis Ford Coppola  $ 4o8,035,783  1.098.714  9,.0   
4              R     Quentin Tarantino    $ 222831817  1.780.147  8,9f   

       original_title_fixed               title_clean  
0  The Shawshank Redemption  the_shawshank_redemption  
1         

In [1024]:
# Reorder 'original_title_fixed' column

# Get current column order
cols = list(df.columns)

# Remove 'original_title_fixed' from its current position
cols.remove('original_title_fixed')

# Find index of 'imdb_title_id'
idx = cols.index('imdb_title_id')

# Insert 'original_title_fixed' right after 'imdb_title_id'
cols.insert(idx + 1, 'original_title_fixed')

# Reorder DataFrame
df = df[cols]

# Rename column 'original_title_fixed' → 'original_title'
df = df.rename(columns={'original_title_fixed': 'original_title'})


### 4.3 release date & release_year

In [1025]:
# 10 'release_year' unique values
print(df['release_year'].unique()[:10])

['1995-02-10' '09 21 1972' ' 23 -07-2008' '1975-09-25' '1994-10-28'
 '22 Feb 04' '1994-03-11' '1957-09-04' '2010-09-24' '10-29-99']


In [1026]:
# Parse 'release_year' column into datetime format
# - errors='coerce' will set invalid parsing results to NaT (Not a Time)
# - infer_datetime_format=True tries to automatically detect the date format
df['release_year_parsed'] = pd.to_datetime(
    df['release_year'], 
    errors='coerce'
)

# Identify rows where parsing failed (NaT values)
# This helps spot problematic or malformed entries in 'release_year'
naT_rows = df[df['release_year_parsed'].isna()]
print(naT_rows[['release_year', 'release_year_parsed']])


                   release_year release_year_parsed
1                    09 21 1972                 NaT
2                   23 -07-2008                 NaT
5                     22 Feb 04                 NaT
9                      10-29-99                 NaT
12       23rd December of 1966                  NaT
15                     01/16-03                 NaT
18                   18/11/1976                 NaT
45                     21-11-46                 NaT
70  The 6th of marzo, year 1951                 NaT
83                   1984-02-34                 NaT
84                   1976-13-24                 NaT


In [1027]:
def fix_date(val):
    """
    Attempt to normalize and parse messy date strings into pandas datetime objects.

    This function handles common irregularities such as extra words ("the", "of", "year"),
    inconsistent month names, and encoding issues. It uses a predefined `month_map` to
    replace non-standard month representations with valid ones, then attempts to parse
    the cleaned string using `dateutil.parser`.

    Parameters
    ----------
    val : str, int, float, or NaN
        The input value representing a date. Can be a string, numeric, or NaN.

    Returns
    -------
    pandas.Timestamp (datetime64) or NaT
        A parsed datetime object if successful.
        Returns pandas.NaT if the input is missing or cannot be parsed.
    """
    if pd.isnull(val):
        return pd.NaT
    s = str(val).strip()

    s = re.sub(r'\b(the|of|year)\b', '', s, flags=re.IGNORECASE).strip()

    for k, v in month_map.items():
        if k.lower() in s.lower():
            s = s.lower().replace(k.lower(), v)

    try:
        dt = parser.parse(s, dayfirst=True, yearfirst=False)
        return dt
    except Exception:
        return pd.NaT

In [1028]:
month_map = {"marzo": "March"}

# manual revision of incorrect data
df.loc[83, 'release_year'] = "December 9, 1983"
df.loc[84, 'release_year'] = "February 8, 1976"


# Add the initial parsing results column
df['release_year_parsed'] = pd.to_datetime(df['release_year'], errors='coerce')

# Fix NaT rows with the fix_date function
mask_nat = df['release_year_parsed'].isna()
df.loc[mask_nat, 'release_year_parsed'] = df.loc[mask_nat, 'release_year'].apply(fix_date)

# Separate the year into the release_year column without decimals
df['release_year'] = df['release_year_parsed'].dt.year.astype('Int64')

# Change column name to release_date
df = df.rename(columns={'release_year_parsed': 'release_date'})

# Result check
print(df[['release_year', 'release_date']].head(20))

    release_year release_date
0           1995   1995-02-10
1           1972   1972-09-21
2           2008   2008-07-23
3           1975   1975-09-25
4           1994   1994-10-28
5           2004   2004-02-22
6           1994   1994-03-11
7           1957   1957-09-04
8           2010   2010-09-24
9           1999   1999-10-29
10          1994   1994-10-06
11          2002   2002-01-18
12          1966   1966-12-23
14          1999   1999-05-07
15          2003   2003-01-16
16          1980   1980-09-19
17          1990   1990-09-20
18          1976   1976-11-18
19          2014   2014-11-06
20          1995   1995-12-15


In [1029]:
# Get current column order
cols = list(df.columns)

# Remove 'release_date' from its current position
# (to avoid duplication when we re-insert it later)
cols.remove('release_date')

# Find the index of 'original_title'
idx = cols.index('original_title')

# Insert 'release_date' right after 'original_title_fixed'
# so it will appear before 'release_year'
cols.insert(idx + 1, 'release_date')

# Reorder DataFrame according to the new column order
df = df[cols]

# Quick check to confirm new order
print(df.head())


  imdb_title_id            original_title release_date  release_year  \
0     tt0111161  The Shawshank Redemption   1995-02-10          1995   
1     tt0068646             The Godfather   1972-09-21          1972   
2     tt0468569           The Dark Knight   2008-07-23          2008   
3     tt0071562    The Godfather: Part II   1975-09-25          1975   
4     tt0110912              Pulp Fiction   1994-10-28          1994   

                  genre duration country content_rating              director  \
0                 Drama      142     USA              R        Frank Darabont   
1          Crime, Drama      175     USA              R  Francis Ford Coppola   
2  Action, Crime, Drama      152      US          PG-13     Christopher Nolan   
3          Crime, Drama      220     USA              R  Francis Ford Coppola   
4          Crime, Drama              USA              R     Quentin Tarantino   

          income      votes score               title_clean  
0     $ 28815245  

## 4.4 genre

In [1030]:
df['genre'].unique()

array(['Drama', 'Crime, Drama', 'Action, Crime, Drama',
       'Action, Adventure, Drama', 'Biography, Drama, History',
       'Action, Adventure, Sci-Fi', 'Drama, Romance', 'Western',
       'Action, Sci-Fi', 'Action, Adventure, Fantasy',
       'Biography, Crime, Drama', 'Adventure, Drama, Sci-Fi',
       'Crime, Drama, Mystery', 'Crime, Drama, Thriller', 'Drama, War',
       'Crime, Drama, Fantasy', 'Animation, Adventure, Family',
       'Comedy, Drama, Romance', 'Comedy, Drama, Thriller',
       'Drama, Family, Fantasy', 'Drama, Mystery, Sci-Fi',
       'Adventure, Comedy, Sci-Fi', 'Crime, Mystery, Thriller',
       'Animation, Adventure, Drama', 'Biography, Comedy, Drama',
       'Biography, Drama, Music', 'Drama, Music',
       'Horror, Mystery, Thriller', 'Drama, Romance, War',
       'Animation, Drama, War', 'Comedy, Drama, Family',
       'Action, Adventure', 'Drama, Western', 'Mystery, Thriller',
       'Drama, Horror', 'Horror, Sci-Fi', 'Drama, Mystery, War',
       'Action,

In [1031]:
def clean_genre(genre_str):
    """
    Normalize and clean a genre string into a standardized comma-separated string.

    This function:
    1. Splits the input string by commas.
    2. Strips leading/trailing whitespace from each genre.
    3. Maps each genre to a standardized name using a predefined `mapping` dictionary.
    4. Removes duplicates and returns a comma-separated string.

    Parameters
    ----------
    genre_str : str
        A string containing one or more genres separated by commas.

    Returns
    -------
    str
        A cleaned, normalized, and unique genre string with genres separated by commas.
        Returns an empty string if the input is NaN or empty.
    """
    if pd.isna(genre_str):
        return ""

    # Split by commas and strip whitespace
    genres = [g.strip() for g in genre_str.split(",")]

    # Normalize genre names using mapping dictionary
    genres = [mapping.get(g, g) for g in genres]

    # Remove duplicates and sort for consistency
    genres = sorted(set(genres))

    # Join back into a comma-separated string
    return ", ".join(genres)


In [1032]:
# Genre name normalization mapping
mapping = {
    "Sci-Fi": "Science Fiction",
    "Biography": "Biographical"
}

# Add the genre_clean column
df["genre_clean"] = df["genre"].apply(clean_genre)

## 4.5 duration

In [1033]:
# --- Clean 'duration' column ---
df['duration_clean'] = (
    df['duration']
    .str.replace(r'[^0-9]', '', regex=True)   # keep only digits
    .replace('', np.nan)                      # empty strings -> NaN
    .astype(float)
)

# Impute missing values with mean

# Calculate mean and round up
duration_mean = df['duration_clean'].mean(skipna=True)
duration_mean_ceil = math.ceil(duration_mean)

# Fill NaN values with the rounded-up mean
df['duration_clean'] = df['duration_clean'].fillna(duration_mean_ceil)

In [1034]:
# Overwrite the old 'duration' column with the cleaned version
df['duration'] = df['duration_clean'].astype(int)

# Drop the temporary 'duration_clean' column
df = df.drop(columns=['duration_clean'])

# Quick check to confirm only one 'duration' column exists
print(df.head())

  imdb_title_id            original_title release_date  release_year  \
0     tt0111161  The Shawshank Redemption   1995-02-10          1995   
1     tt0068646             The Godfather   1972-09-21          1972   
2     tt0468569           The Dark Knight   2008-07-23          2008   
3     tt0071562    The Godfather: Part II   1975-09-25          1975   
4     tt0110912              Pulp Fiction   1994-10-28          1994   

                  genre  duration country content_rating  \
0                 Drama       142     USA              R   
1          Crime, Drama       175     USA              R   
2  Action, Crime, Drama       152      US          PG-13   
3          Crime, Drama       220     USA              R   
4          Crime, Drama       136     USA              R   

               director         income      votes score  \
0        Frank Darabont     $ 28815245  2.278.845   9.3   
1  Francis Ford Coppola    $ 246120974  1.572.674   9.2   
2     Christopher Nolan   $ 1

## 4.6 country

In [1035]:
def basic_country_clean(x):
    """
    Clean and normalize country name strings by removing common artifacts.

    This function:
    1. Returns NaN values unchanged.
    2. Strips leading and trailing whitespace.
    3. Removes periods from the string.
    4. Removes trailing numeric characters (e.g., "Italy1" → "Italy").

    Parameters
    ----------
    x : str or NaN
        The input country name string to clean. Can be a string or NaN.

    Returns
    -------
    str or NaN
        A cleaned country name string with whitespace, periods, and trailing
        numbers removed. Returns NaN if the input is NaN.
    """
    if pd.isna(x):
        return x
    
    x = x.strip()
    x = x.replace('.', '')
    
    # Remove trailing numbers (e.g., "Italy1" → "Italy")
    x = re.sub(r'\d+$', '', x)
    
    return x


In [1036]:
# --- Step 1: Clean and replace 'country' column in place ---
country_mapping = {
    'US': 'United States',
    'USA': 'United States',
    'UK': 'United Kingdom',
    'New Zesland': 'New Zealand',
    'New Zeland': 'New Zealand',
    'West Germany': 'Germany'
}

# Apply basic cleaning
df['country'] = df['country'].apply(basic_country_clean)

# Apply mapping standardization
df['country'] = df['country'].replace(country_mapping)

# Apply Title Case formatting
df['country'] = df['country'].str.title()


# --- Step 2: Create 'country_clean' in snake_case ---
df['country_clean'] = (
    df['country']
    .str.lower()          # lowercase
    .str.replace(' ', '_')  # replace spaces with underscores
)

# Quick check
print(df[['country', 'country_clean']].head())


         country  country_clean
0  United States  united_states
1  United States  united_states
2  United States  united_states
3  United States  united_states
4  United States  united_states


## 4.7 content_rating

In [1037]:
# Standardizing unrated with 'Not Rated'
df['content_rating'] = df['content_rating'].replace({
    'Unrated': 'Not Rated',
    'Approved': 'Not Rated'
})

# Fill NaN with 'Not Rated'
df['content_rating'] = df['content_rating'].fillna('Not Rated')

## 4.8 director

In [1038]:
df['director'].unique()

array(['Frank Darabont', 'Francis Ford Coppola', 'Christopher Nolan',
       'Quentin Tarantino', 'Peter Jackson', 'Steven Spielberg',
       'Sidney Lumet', 'David Fincher', 'Robert Zemeckis', 'Sergio Leone',
       'Lana Wachowski, Lilly Wachowski', 'Irvin Kershner',
       'Martin Scorsese', 'Milos Forman', 'Jonathan Demme',
       'George Lucas', 'Fernando Meirelles, KÃ¡tia Lund',
       'Hayao Miyazaki', 'Roberto Benigni', 'Bong Joon Ho', 'Frank Capra',
       'Akira Kurosawa', 'Ridley Scott', 'Tony Kaye', 'Luc Besson',
       'James Cameron', 'Bryan Singer', 'Roger Allers, Rob Minkoff',
       'Todd Phillips', 'Olivier Nakache, Ã\x89ric Toledano',
       'Roman Polanski', 'Damien Chazelle', 'Alfred Hitchcock',
       'Michael Curtiz', 'Isao Takahata', 'Giuseppe Tornatore',
       'Charles Chaplin', 'Andrew Stanton', 'Stanley Kubrick',
       'Anthony Russo, Joe Russo', 'Chan-wook Park',
       'Lee Unkrich, Adrian Molina', 'Florian Henckel von Donnersmarck',
       'Bob Persichet

In [1039]:
# copy original column
df['director_original'] = df['director']

In [1040]:
# Fix mojibake encoding (KÃ¡tia → Kátia)
def fix_mojibake(text):
    """
    Attempt to fix mojibake (garbled text caused by encoding issues) by re-encoding.

    This function tries to re-interpret a string that was incorrectly decoded
    from UTF-8 as Latin-1. It re-encodes the input as 'latin1' and then decodes
    it back to 'utf-8', which often resolves common mojibake problems.

    Parameters
    ----------
    text : str
        The input text string that may contain mojibake artifacts.

    Returns
    -------
    str
        A cleaned string with encoding issues fixed if possible.
        If the input is not a string or cannot be re-encoded/decoded,
        the original input is returned unchanged.
    """
    try:
        return text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError, AttributeError):
        return text

In [1041]:
# Clean director_original (human-readable, keep accent)
def clean_director_original(name):
    """
    Clean and normalize director name strings by fixing encoding 
    issues and removing unwanted characters.

    This function:
    1. Returns NaN values unchanged.
    2. Fixes common mojibake (encoding issues) using `fix_mojibake`.
    3. Strips leading and trailing whitespace.
    4. Removes unwanted symbols, keeping only:
       - Letters (including accented characters)
       - Spaces
       - Periods
       - Hyphens
       - Apostrophes
       - Commas
    5. Normalizes multiple spaces into a single space.

    Parameters
    ----------
    name : str or NaN
        The input director name string to clean. Can be a string or NaN.

    Returns
    -------
    str or NaN
        A cleaned director name string with encoding issues fixed, 
        unwanted symbols removed, and spacing normalized. 
        Returns NaN if the input is NaN.
    """
    if pd.isna(name):
        return name
    
    # Fix encoding issues (e.g., mojibake)
    name = fix_mojibake(name)
    
    # Strip leading/trailing spaces
    name = name.strip()
    
    # Remove unwanted symbols, keep only allowed characters
    name = re.sub(r"[^A-Za-zÀ-ÖØ-öø-ÿ\s\.\-',]", "", name)
    
    # Normalize multiple spaces
    name = re.sub(r"\s+", " ", name)
    
    return name

In [1042]:
# applying clean_director_original function

df['director_original'] = df['director_original'].apply(clean_director_original)

In [1043]:
# Helper: remove accents
def remove_accents(text):
    """
    Remove diacritical marks (accents) from characters in a string.

    This function uses Unicode normalization (NFKD) to decompose accented
    characters into their base character plus combining marks, then filters
    out the combining marks. The result is a plain ASCII-like string without
    accents.

    Parameters
    ----------
    text : str
        The input string that may contain accented characters.

    Returns
    -------
    str
        A new string with all accents removed. For example,
        "Café" → "Cafe", "São Paulo" → "Sao Paulo".
    """
    return ''.join(
        char for char in unicodedata.normalize('NFKD', text)
        if not unicodedata.combining(char)
    )

In [1044]:
# Create director_normalized
def clean_director_normalized(name):
    """
    Normalize director name strings for consistent machine-friendly representation.

    This function:
    1. Returns NaN values unchanged.
    2. Removes diacritical marks (accents) using `remove_accents`.
    3. Converts the string to lowercase.
    4. Removes all non-letter characters except spaces.
    5. Normalizes multiple spaces into a single space and strips leading/trailing spaces.

    Parameters
    ----------
    name : str or NaN
        The input director name string to normalize. Can be a string or NaN.

    Returns
    -------
    str or NaN
        A normalized director name string in lowercase, without accents or
        unwanted symbols, and with consistent spacing. Returns NaN if the input is NaN.

    Examples
    --------
    >>> clean_director_normalized("François Ozon")
    'francois ozon'
    >>> clean_director_normalized("  Steven Spielberg!! ")
    'steven spielberg'
    """
    if pd.isna(name):
        return name
    
    name = remove_accents(name)
    name = name.lower()
    
    # Keep only letters and spaces
    name = re.sub(r"[^a-z\s]", "", name)
    
    # Normalize spaces
    name = re.sub(r"\s+", " ", name).strip()

    # Convert spaces to underscores (snake_case)
    name = name.replace(" ", "_")
    
    return name

In [1045]:
# applying clean_director_normalized function

df['director_normalized'] = df['director_original'].apply(clean_director_normalized)

# Replace the data in 'director' with values from 'director_original'
df['director'] = df['director_original']

# Drop 'director_original' column
df = df.drop(columns=['director_original'])

# Quick check
print(df.head())


  imdb_title_id            original_title release_date  release_year  \
0     tt0111161  The Shawshank Redemption   1995-02-10          1995   
1     tt0068646             The Godfather   1972-09-21          1972   
2     tt0468569           The Dark Knight   2008-07-23          2008   
3     tt0071562    The Godfather: Part II   1975-09-25          1975   
4     tt0110912              Pulp Fiction   1994-10-28          1994   

                  genre  duration        country content_rating  \
0                 Drama       142  United States              R   
1          Crime, Drama       175  United States              R   
2  Action, Crime, Drama       152  United States          PG-13   
3          Crime, Drama       220  United States              R   
4          Crime, Drama       136  United States              R   

               director         income      votes score  \
0        Frank Darabont     $ 28815245  2.278.845   9.3   
1  Francis Ford Coppola    $ 246120974  1.572.

## 4.9 income

In [1046]:
df['income'].unique()

array(['$ 28815245', '$ 246120974', '$ 1005455211', '$ 4o8,035,783',
       '$ 222831817', '$ 1142271098', '$ 322287794', '$ 576',
       '$ 869784991', '$ 101218804', '$ 678229452', '$ 887934303',
       '$ 25252481', '$ 465718588', '$ 951227416', '$ 549265501',
       '$ 46879633', '$ 108997629', '$ 696742056', '$ 327333559',
       '$ 272753884', '$ 775768912', '$ 482349603', '$ 286801374',
       '$ 30680793', '$ 355467056', '$ 230098753', '$ 257604912',
       '$ 6130720', '$ 322773', '$ 465361176', '$ 291465034',
       '$ 109676311', '$ 388774684', '$ 23875127', '$ 19552639',
       '$ 520884847', '$ 23341568', '$ 968511805', '$ 1074251311',
       '$ 426588510', '$ 120072577', '$ 48983260', '$ 32008644',
       '$ 4374761', '$ 112911', '$ 516962', '$ 13826605', '$ 457688',
       '$ 1081133191', '$ 425368238', '$ 39970386', '$ 521311860',
       '$ 46520613', '$ 390133212', '$ 2048359754', '$ 108110316',
       '$ 2797800564', '$ 91968688', '$ 15002116', '$ 9443876',
       '$ 

In [1047]:
# --- Clean 'income' column ---
df['income_clean'] = (
    df['income']
    .str.replace('o', '0', regex=False)       # replace letter o with zero
    .str.replace(r'[^0-9]', '', regex=True)   # keep only digits
    .replace('', np.nan)                      # empty strings -> NaN
    .astype(int)
)

# --- Check results ---
print(df[['income', 'income_clean']].head())

          income  income_clean
0     $ 28815245      28815245
1    $ 246120974     246120974
2   $ 1005455211    1005455211
3  $ 4o8,035,783     408035783
4    $ 222831817     222831817


In [1048]:
# Replace the data in 'income' with values from 'income_clean'
df['income'] = df['income_clean']

# Drop 'income_clean' column
df = df.drop(columns=['income_clean'])

# Quick check
print(df.head())

  imdb_title_id            original_title release_date  release_year  \
0     tt0111161  The Shawshank Redemption   1995-02-10          1995   
1     tt0068646             The Godfather   1972-09-21          1972   
2     tt0468569           The Dark Knight   2008-07-23          2008   
3     tt0071562    The Godfather: Part II   1975-09-25          1975   
4     tt0110912              Pulp Fiction   1994-10-28          1994   

                  genre  duration        country content_rating  \
0                 Drama       142  United States              R   
1          Crime, Drama       175  United States              R   
2  Action, Crime, Drama       152  United States          PG-13   
3          Crime, Drama       220  United States              R   
4          Crime, Drama       136  United States              R   

               director      income      votes score  \
0        Frank Darabont    28815245  2.278.845   9.3   
1  Francis Ford Coppola   246120974  1.572.674   9.2

## 4.10 votes

In [1049]:
df['votes_clean'] = (
    df['votes']
    .str.replace(r'\.', '', regex=True)       # remove thousand separators
    .replace('', np.nan)
    .astype(float)
    .astype('Int64')                          # integer with NaN support
)

# --- Check results ---
print(df[['votes', 'votes_clean']].head())

       votes  votes_clean
0  2.278.845      2278845
1  1.572.674      1572674
2  2.241.615      2241615
3  1.098.714      1098714
4  1.780.147      1780147


In [1050]:
# Replace the data in 'votes' with values from 'votes_clean'
df['votes'] = df['votes_clean']

# Drop 'votes_clean' column
df = df.drop(columns=['votes_clean'])

# Quick check
print(df.head())

  imdb_title_id            original_title release_date  release_year  \
0     tt0111161  The Shawshank Redemption   1995-02-10          1995   
1     tt0068646             The Godfather   1972-09-21          1972   
2     tt0468569           The Dark Knight   2008-07-23          2008   
3     tt0071562    The Godfather: Part II   1975-09-25          1975   
4     tt0110912              Pulp Fiction   1994-10-28          1994   

                  genre  duration        country content_rating  \
0                 Drama       142  United States              R   
1          Crime, Drama       175  United States              R   
2  Action, Crime, Drama       152  United States          PG-13   
3          Crime, Drama       220  United States              R   
4          Crime, Drama       136  United States              R   

               director      income    votes score               title_clean  \
0        Frank Darabont    28815245  2278845   9.3  the_shawshank_redemption   
1  F

## 4.11 score

In [1051]:
# First fix known problematic cases by index
fix_map = {
    4: '8.9',   # '8,9f'
    10: '8.8',  # '8:8'
    16: '8.7',  # '8,7e-0'
    21: '8.6'   # '8,6'
}
for idx, val in fix_map.items():
    if idx in df.index:
        df.loc[idx, 'score'] = val

# Now clean the rest
df['score_clean'] = (
    df['score']
    .str.replace(r'[^0-9\.]', '', regex=True) # keep digits and dots
    .str.replace(r'\.+', '.', regex=True)     # collapse multiple dots
    .str.strip('.')                           # remove leading/trailing dots
    .replace('', np.nan)                      # empty strings -> NaN
    .astype(float)
)

# Ensure scores are between 0 and 10
df.loc[(df['score_clean'] < 0) | (df['score_clean'] > 10), 'score_clean'] = np.nan

# --- Check results ---
print(df[['score', 'score_clean']].head(25))

    score  score_clean
0     9.3          9.3
1     9.2          9.2
2      9.          9.0
3    9,.0          9.0
4     8.9          8.9
5    08.9          8.9
6     8.9          8.9
7     8.9          8.9
8    8..8          8.8
9     8.8          8.8
10    8.8          8.8
11    8.8          8.8
12    8.8          8.8
14  ++8.7          8.7
15   8.7.          8.7
16    8.7          8.7
17    8.7          8.7
18    8.7          8.7
19    8.6          8.6
20    8.6          8.6
21    8.6          8.6
22    8.6          8.6
23    8.6          8.6
24    8.6          8.6
25    8.6          8.6


In [1052]:
# Replace the data in 'score' with values from 'score_clean'
df['score'] = df['score_clean']

# Drop 'score_clean' column
df = df.drop(columns=['score_clean'])

# Quick check
print(df.head())

  imdb_title_id            original_title release_date  release_year  \
0     tt0111161  The Shawshank Redemption   1995-02-10          1995   
1     tt0068646             The Godfather   1972-09-21          1972   
2     tt0468569           The Dark Knight   2008-07-23          2008   
3     tt0071562    The Godfather: Part II   1975-09-25          1975   
4     tt0110912              Pulp Fiction   1994-10-28          1994   

                  genre  duration        country content_rating  \
0                 Drama       142  United States              R   
1          Crime, Drama       175  United States              R   
2  Action, Crime, Drama       152  United States          PG-13   
3          Crime, Drama       220  United States              R   
4          Crime, Drama       136  United States              R   

               director      income    votes  score               title_clean  \
0        Frank Darabont    28815245  2278845    9.3  the_shawshank_redemption   
1 

In [1053]:
print(df.head())

  imdb_title_id            original_title release_date  release_year  \
0     tt0111161  The Shawshank Redemption   1995-02-10          1995   
1     tt0068646             The Godfather   1972-09-21          1972   
2     tt0468569           The Dark Knight   2008-07-23          2008   
3     tt0071562    The Godfather: Part II   1975-09-25          1975   
4     tt0110912              Pulp Fiction   1994-10-28          1994   

                  genre  duration        country content_rating  \
0                 Drama       142  United States              R   
1          Crime, Drama       175  United States              R   
2  Action, Crime, Drama       152  United States          PG-13   
3          Crime, Drama       220  United States              R   
4          Crime, Drama       136  United States              R   

               director      income    votes  score               title_clean  \
0        Frank Darabont    28815245  2278845    9.3  the_shawshank_redemption   
1 

In [1054]:
# Export DataFrame to Excel in ../data/interim folder
output_path = "../data/interim/cleaned_data2.xlsx"
df.to_excel(output_path, sheet_name="Cleaned", index=False)

In [1055]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 100
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   imdb_title_id        100 non-null    object        
 1   original_title       100 non-null    object        
 2   release_date         100 non-null    datetime64[ns]
 3   release_year         100 non-null    Int64         
 4   genre                100 non-null    object        
 5   duration             100 non-null    int64         
 6   country              100 non-null    object        
 7   content_rating       100 non-null    object        
 8   director             100 non-null    object        
 9   income               100 non-null    int64         
 10  votes                100 non-null    Int64         
 11  score                100 non-null    float64       
 12  title_clean          100 non-null    object        
 13  genre_clean          100 non-null    obj

# 5. Feature Engineering
- Create new features if needed.
- Encode categorical variables.
- Normalize or standardize numerical features.

In [1056]:
df['genre_clean']

0                           Drama
1                    Crime, Drama
2            Action, Crime, Drama
3                    Crime, Drama
4                    Crime, Drama
                  ...            
96           Comedy, Crime, Drama
97     Adventure, Drama, Thriller
98                        Western
99                          Drama
100      Comedy, Musical, Romance
Name: genre_clean, Length: 100, dtype: object

In [1057]:
# Convert to list of labels
df['genre_clean'] = df['genre_clean'].str.split(', ')

# reset index
df = df.reset_index(drop=True)

# One-Hot Encoding for genre
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df["genre_clean"])
df_ohe_genre = pd.DataFrame(genre_encoded, columns=mlb.classes_)

In [1058]:
# Overwrite the old 'genre' column with the cleaned version
df['genre'] = df['genre_clean'].str.join(', ')

# Quick check to confirm only one 'genre' column exists
print(df.head())

  imdb_title_id            original_title release_date  release_year  \
0     tt0111161  The Shawshank Redemption   1995-02-10          1995   
1     tt0068646             The Godfather   1972-09-21          1972   
2     tt0468569           The Dark Knight   2008-07-23          2008   
3     tt0071562    The Godfather: Part II   1975-09-25          1975   
4     tt0110912              Pulp Fiction   1994-10-28          1994   

                  genre  duration        country content_rating  \
0                 Drama       142  United States              R   
1          Crime, Drama       175  United States              R   
2  Action, Crime, Drama       152  United States          PG-13   
3          Crime, Drama       220  United States              R   
4          Crime, Drama       136  United States              R   

               director      income    votes  score               title_clean  \
0        Frank Darabont    28815245  2278845    9.3  the_shawshank_redemption   
1 

# 6. Final Cleaned Dataset
- Save the cleaned dataset.
- Provide a summary of cleaning steps applied.

In [1059]:
# --- 1. Export cleaned_for_report ---
cols_report = [
    "imdb_title_id", "original_title", "release_date", "release_year",
    "genre", "duration", "country", "content_rating", "director",
    "income", "votes", "score"
]

df_report = df[cols_report]
df_report.to_csv("../data/processed/cleaned_for_report.csv", index=False)


# --- 2. Export cleaned_for_analysis ---
# Select and rename columns
df_analysis = df[[
    "imdb_title_id", "title_clean", "release_date", "release_year",
    "duration", "country_clean", "content_rating", "director_normalized",
    "income", "votes", "score"
]].rename(columns={
    "title_clean": "title",
    "country_clean": "country",
    "director_normalized": "director"
})

# Concatenate with one-hot encoded genre DataFrame
df_analysis = pd.concat([df_analysis, df_ohe_genre], axis=1)

df_analysis.to_csv("../data/processed/cleaned_for_analysis.csv", index=False)